In [1]:
#Import Relevant Libraries
import random
import numpy as np
import cvxpy as cp

from qiskit import QuantumCircuit, execute, Aer, IBMQ, BasicAer
from qiskit.visualization import plot_histogram, plot_state_qsphere as plot_q
from qiskit.quantum_info import Statevector

from qiskit import quantum_info as qi
from toqito.channel_metrics import channel_fidelity

from toqito.state_metrics import fidelity
from toqito.channels import partial_trace as pt
from qiskit.quantum_info import DensityMatrix, state_fidelity, partial_trace

In [6]:
def rand_numb(low_in,high_in):
    '''
    Returns a random number between low_in and high_in, including both end points.
    '''
    return np.random.uniform(low=low_in, high=np.nextafter(high_in, np.inf))

In [7]:
'''
Defines parameters that specify the unitary extension of the two channels.
'''
unitary_param_1 = [0.65523857, 5.36749967, 4.08036963, 0.83121276]
unitary_param_2 = [1.28317084, 1.99895141, 4.21274631, 1.10246348]

In [8]:
'''
Create unitary extension of Channel 1
'''
size = 2
qc1 = QuantumCircuit(size)
for j in range(2):
    for i in range(size):
        qc1.rx(unitary_param_1[i+j*size], i)

    for i in range(size-1):
        qc1.cx(i, i+1)

qc1.draw()

┌─────────────┐      ┌────────────┐     
q_0: ┤ Rx(0.65524) ├──■───┤ Rx(4.0804) ├──■──
     └┬────────────┤┌─┴─┐┌┴────────────┤┌─┴─┐
q_1: ─┤ Rx(5.3675) ├┤ X ├┤ Rx(0.83121) ├┤ X ├
      └────────────┘└───┘└─────────────┘└───┘

In [12]:
'''
Create unitary extension of Channel 2
'''
size = 2
qc2 = QuantumCircuit(size)
for j in range(2):
    for i in range(size):
        qc2.rx(unitary_param_2[i+j*size], i)

    for i in range(size-1):
        qc2.cx(i, i+1)

qc2.draw()

┌────────────┐     ┌────────────┐     
q_0: ┤ Rx(1.2832) ├──■──┤ Rx(4.2127) ├──■──
     ├───────────┬┘┌─┴─┐├────────────┤┌─┴─┐
q_1: ┤ Rx(1.999) ├─┤ X ├┤ Rx(1.1025) ├┤ X ├
     └───────────┘ └───┘└────────────┘└───┘

In [13]:
'''
Creates extension of the Choi State of channel 1
'''
qua = QuantumCircuit(3)
qua.h(1)
qua.cx(1, 2)
qua.append(qc1, [0, 1])
qua.draw()

┌────────────┐
q_0: ──────────┤0           ├
     ┌───┐     │  circuit-3 │
q_1: ┤ H ├──■──┤1           ├
     └───┘┌─┴─┐└────────────┘
q_2: ─────┤ X ├──────────────
          └───┘

In [14]:
'''
Creates extension of the Choi State of channel 2
'''
qua2 = QuantumCircuit(3)
qua2.h(1)
qua2.cx(1, 2)
qua2.append(qc2, [0, 1])
qua2.draw()

┌────────────┐
q_0: ──────────┤0           ├
     ┌───┐     │  circuit-8 │
q_1: ┤ H ├──■──┤1           ├
     └───┘┌─┴─┐└────────────┘
q_2: ─────┤ X ├──────────────
          └───┘

In [15]:
'''
Creates Choi matrices of the two channels
'''
psi_1 = np.array(DensityMatrix.from_instruction(qua.reverse_bits()))
psi_2 = np.array(DensityMatrix.from_instruction(qua2.reverse_bits()))

rho_1 = pt(psi_1, [1], [2, 2, 2])
rho_2 = pt(psi_2, [1], [2, 2, 2])

In [16]:
'''
Multiply the choi matrix by the normalization
'''
choi_1 = np.array(2*rho_1)
choi_2 = np.array(2*rho_2)

In [17]:
'''
Primal Formulation
'''
if choi_1.shape != choi_2.shape:
    raise ValueError("The Choi matrices provided should be of equal dimension.")

choi_dim_x, choi_dim_y = choi_1.shape

if choi_dim_x != choi_dim_y:
    raise ValueError("The Choi matrix provided must be square.")

choi_dim = choi_dim_x
dim = int(choi_dim/2)

L = cp.Variable()
Q = cp.Variable((choi_dim, choi_dim), complex=True)
G = cp.bmat([[choi_1, Q.H],[Q, choi_2]])
c = [np.identity(dim)* L >>0, np.identity(dim) * L << cp.real(pt(Q, [1], [dim, dim])), G>>0]
obj = cp.Maximize(cp.real(L))
problem = cp.Problem(obj, constraints = c)
print("Root Fidelity is : " + str(problem.solve()))

Root Fidelity is : 0.8442359418114711


In [18]:
'''
Dual Formulation
'''
if choi_1.shape != choi_2.shape:
    raise ValueError("The Choi matrices provided should be of equal dimension.")

choi_dim_x, choi_dim_y = choi_1.shape

if choi_dim_x != choi_dim_y:
    raise ValueError("The Choi matrix provided must be square.")

choi_dim = choi_dim_x
dim = int(choi_dim/2)

rho = cp.Variable((dim, dim))
W = cp.Variable((choi_dim, choi_dim),complex = True)
Z = cp.Variable((choi_dim, choi_dim),complex = True)

#Is this correct?
G = cp.bmat([[W, cp.kron(np.identity(dim), rho)], [cp.kron(np.identity(dim),rho),Z]])

c = [rho>>0, cp.trace(rho)==1, G>>0]

obj = cp.Minimize(cp.real(cp.trace(choi_1@W) + cp.trace(choi_2@Z)))

prob = cp.Problem(obj,constraints = c)

prob.solve(verbose=False)

print(0.5*prob.value)

WARN: A->p (column pointers) not strictly increasing, column 16 empty
WARN: A->p (column pointers) not strictly increasing, column 21 empty
WARN: A->p (column pointers) not strictly increasing, column 26 empty
WARN: A->p (column pointers) not strictly increasing, column 31 empty
WARN: A->p (column pointers) not strictly increasing, column 48 empty
WARN: A->p (column pointers) not strictly increasing, column 53 empty
WARN: A->p (column pointers) not strictly increasing, column 58 empty
WARN: A->p (column pointers) not strictly increasing, column 63 empty
0.8441893585703406
